# * VINSIGHT : Revenue
    Postpaid Revenue : TMH(Jan-Jun 2024) ใช้ data พี่เป้า(Definition เก่า ตัวเลขใกล้เคียงกัน)

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000100GEO' --Total Revenue (Geo)
        , 'DB0R000100GEO' --Total Revenue (Geo) : DTAC
        , 'TB0R000100GEO' --Total Revenue (Geo) : TRUE

        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B0R000101GEO' --Mobile Revenue (Geo)
        , 'DB0R000101GEO' --Mobile Revenue (Geo) : DTAC
        , 'TB0R000101GEO' --Mobile Revenue (Geo) : TMH
        
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
         
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'B2R020100' --Postpaid Revenue B2B
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'DB2R020100' --Postpaid Revenue B2B : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'TB2R020100' --Postpaid Revenue B2B : TMH
        
        , 'TB3R000100' --TOL Revenue
        
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-08-13, 12:33:37

DataFrame: 15340 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview(Corp) by Period

In [4]:
''' Revenue(Corp) Yearly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_yearly_df = rev_corp_yearly_df.loc[rev_corp_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_yearly_df['TOTAL(Corp)'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B0R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['PRE'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB1R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['PRE_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB1R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['POST'] = np.where(rev_corp_yearly_df['METRIC_CD']=='B2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_T'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB2R000100', rev_corp_yearly_df['C'], 0)
# rev_corp_yearly_df['POST_D'] = np.where(rev_corp_yearly_df['METRIC_CD']=='DB2R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TOL'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB3R000100', rev_corp_yearly_df['C'], 0)
rev_corp_yearly_df['TVS'] = np.where(rev_corp_yearly_df['METRIC_CD']=='TB4R000100', rev_corp_yearly_df['C'], 0)

rev_corp_yearly_df = rev_corp_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_yearly_df['YoY'] = rev_corp_yearly_df['TOTAL(Corp)'] - rev_corp_yearly_df['TOTAL(Corp)'].shift()
rev_corp_yearly_df = rev_corp_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_corp_yearly_df = rev_corp_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_yearly_df_display = rev_corp_yearly_df.copy()
rev_corp_yearly_df_display['ACTUAL_AS_OF'] = rev_corp_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_yearly_df_display[col] = rev_corp_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Corp),PRE,POST,TOL,TVS
0,2024,2025-08-13 05:17:31,20241231,0,"149,825,803,306","48,319,930,436","78,295,748,348","18,827,249,425","4,018,244,147"
1,2025,2025-08-13 05:17:31,20250811,"-57,711,607,278","92,114,196,028","30,550,947,852","47,376,398,551","11,845,278,491","2,061,969,199"


In [5]:
''' Revenue(Corp) Quarterly '''

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].copy()
rev_corp_quarterly_df = rev_corp_quarterly_df.loc[rev_corp_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_quarterly_df['TOTAL(Corp)'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B0R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['PRE'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB1R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['PRE_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB1R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['POST'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='B2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_T'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB2R000100', rev_corp_quarterly_df['C'], 0)
# rev_corp_quarterly_df['POST_D'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='DB2R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TOL'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB3R000100', rev_corp_quarterly_df['C'], 0)
rev_corp_quarterly_df['TVS'] = np.where(rev_corp_quarterly_df['METRIC_CD']=='TB4R000100', rev_corp_quarterly_df['C'], 0)

rev_corp_quarterly_df = rev_corp_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_quarterly_df['QoQ'] = rev_corp_quarterly_df['TOTAL(Corp)'] - rev_corp_quarterly_df['TOTAL(Corp)'].shift()
rev_corp_quarterly_df = rev_corp_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_corp_quarterly_df = rev_corp_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_quarterly_df_display = rev_corp_quarterly_df.copy()
rev_corp_quarterly_df_display['ACTUAL_AS_OF'] = rev_corp_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_quarterly_df_display[col] = rev_corp_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Corp),PRE,POST,TOL,TVS
0,20241,2025-08-13 05:17:31,20240331,0,"37,260,216,440","11,842,176,786","19,753,202,663","4,569,920,004","1,013,212,668"
1,20242,2025-08-13 05:17:31,20240630,"-64,093,390","37,196,123,050","11,858,030,293","19,630,569,282","4,665,781,597","968,583,208"
2,20243,2025-08-13 05:17:31,20240930,"428,367,688","37,624,490,738","12,164,985,004","19,459,112,025","4,803,785,483","1,110,012,430"
3,20244,2025-08-13 05:17:31,20241231,"120,482,340","37,744,973,078","12,454,738,354","19,452,864,379","4,787,762,341","926,435,841"
4,20251,2025-08-13 05:17:31,20250331,"-221,910,719","37,523,062,359","12,375,356,967","19,368,310,563","4,810,187,607","883,137,861"
5,20252,2025-08-13 05:17:31,20250630,"-85,327,461","37,437,734,898","12,349,913,007","19,312,243,921","4,833,931,536","860,149,762"
6,20253,2025-08-13 05:17:31,20250811,"-20,284,336,126","17,153,398,772","5,825,677,878","8,695,844,066","2,201,159,349","318,681,577"


In [6]:
''' Revenue(Corp) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_corp_monthly_df = rev_corp_monthly_df.loc[rev_corp_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_monthly_df['TOTAL(Corp)'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B0R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['PRE'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB1R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['PRE_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB1R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['POST'] = np.where(rev_corp_monthly_df['METRIC_CD']=='B2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_T'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB2R000100', rev_corp_monthly_df['C'], 0)
# rev_corp_monthly_df['POST_D'] = np.where(rev_corp_monthly_df['METRIC_CD']=='DB2R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TOL'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB3R000100', rev_corp_monthly_df['C'], 0)
rev_corp_monthly_df['TVS'] = np.where(rev_corp_monthly_df['METRIC_CD']=='TB4R000100', rev_corp_monthly_df['C'], 0)

rev_corp_monthly_df = rev_corp_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_monthly_df['MoM'] = rev_corp_monthly_df['TOTAL(Corp)'] - rev_corp_monthly_df['TOTAL(Corp)'].shift()
rev_corp_monthly_df = rev_corp_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_corp_monthly_df = rev_corp_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_monthly_df_display = rev_corp_monthly_df.copy()
rev_corp_monthly_df_display['ACTUAL_AS_OF'] = rev_corp_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_monthly_df_display[col] = rev_corp_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Corp),PRE,POST,TOL,TVS
0,202401,2025-08-13 05:17:31,20240131,0,"12,527,775,340","4,014,674,524","6,633,797,001","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,20240229,"-279,732,061","12,248,043,279","3,796,713,842","6,572,934,091","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,20240331,"236,354,542","12,484,397,821","4,030,788,420","6,546,471,571","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,20240430,"-125,805,023","12,358,592,798","3,883,185,086","6,572,828,730","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,20240531,"107,213,556","12,465,806,354","4,042,374,933","6,520,016,738","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,20240630,"-94,082,457","12,371,723,897","3,932,470,274","6,537,723,813","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,20240731,"118,155,942","12,489,879,839","4,072,664,861","6,484,830,524","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,20240831,"134,708,079","12,624,587,918","4,109,958,930","6,483,049,467","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,20240930,"-114,564,937","12,510,022,981","3,982,361,213","6,491,232,034","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,20241031,"59,487,568","12,569,510,549","4,128,174,548","6,485,731,903","1,598,256,224","311,443,243"


In [7]:
''' Revenue(Corp) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_weekly_df = rev_corp_weekly_df.loc[rev_corp_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_weekly_df['TOTAL(Corp)'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B0R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['PRE'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB1R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['PRE_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB1R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['POST'] = np.where(rev_corp_weekly_df['METRIC_CD']=='B2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_T'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB2R000100', rev_corp_weekly_df['C'], 0)
# rev_corp_weekly_df['POST_D'] = np.where(rev_corp_weekly_df['METRIC_CD']=='DB2R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TOL'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB3R000100', rev_corp_weekly_df['C'], 0)
rev_corp_weekly_df['TVS'] = np.where(rev_corp_weekly_df['METRIC_CD']=='TB4R000100', rev_corp_weekly_df['C'], 0)

rev_corp_weekly_df = rev_corp_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_weekly_df['WoW'] = rev_corp_weekly_df['TOTAL(Corp)'] - rev_corp_weekly_df['TOTAL(Corp)'].shift()
rev_corp_weekly_df = rev_corp_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_corp_weekly_df = rev_corp_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_weekly_df_display = rev_corp_weekly_df.copy()
rev_corp_weekly_df_display['ACTUAL_AS_OF'] = rev_corp_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_corp_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_weekly_df_display[col] = rev_corp_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Corp),PRE,POST,TOL,TVS
0,2025027,2025-08-13 05:17:31,20250706,0,"2,816,435,575","905,642,039","1,500,488,615","366,697,791","39,023,266"
1,2025028,2025-08-13 05:17:31,20250713,"48,340,895","2,864,776,470","921,096,331","1,491,540,554","403,051,760","45,661,384"
2,2025029,2025-08-13 05:17:31,20250720,"-285,615,905","2,579,160,565","895,004,220","1,308,037,944","334,381,824","38,482,280"
3,2025030,2025-08-13 05:17:31,20250727,"13,397,510","2,592,558,075","900,082,966","1,323,807,710","328,551,447","37,285,184"
4,2025031,2025-08-13 05:17:31,20250803,"728,209,721","3,320,767,795","1,085,791,764","1,706,493,492","393,713,419","125,454,438"
5,2025032,2025-08-13 05:17:31,20250810,"-547,722,199","2,773,045,596","993,009,200","1,365,475,751","374,763,109","32,775,024"
6,2025033,2025-08-13 05:17:31,20250811,"-2,566,390,899","206,654,697","125,051,358",0,0,0


In [8]:
''' Revenue(Corp) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'B1R000100' #Prepaid Revenue
    # , 'DB1R000100' #Prepaid Revenue : DTAC
    # , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R000100' #Postpaid Revenue
    # , 'DB2R000100' #Postpaid Revenue : DTAC
    # , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_corp_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_corp_daily_df = rev_corp_daily_df.loc[rev_corp_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_corp_daily_df['TOTAL(Corp)'] = np.where(rev_corp_daily_df['METRIC_CD']=='B0R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['PRE'] = np.where(rev_corp_daily_df['METRIC_CD']=='B1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB1R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['PRE_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB1R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['POST'] = np.where(rev_corp_daily_df['METRIC_CD']=='B2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_T'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB2R000100', rev_corp_daily_df['C'], 0)
# rev_corp_daily_df['POST_D'] = np.where(rev_corp_daily_df['METRIC_CD']=='DB2R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TOL'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB3R000100', rev_corp_daily_df['C'], 0)
rev_corp_daily_df['TVS'] = np.where(rev_corp_daily_df['METRIC_CD']=='TB4R000100', rev_corp_daily_df['C'], 0)

rev_corp_daily_df = rev_corp_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Corp)':'sum', 'PRE':'sum', 'POST':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_corp_daily_df['DoD'] = rev_corp_daily_df['TOTAL(Corp)'] - rev_corp_daily_df['TOTAL(Corp)'].shift()
rev_corp_daily_df = rev_corp_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_corp_daily_df = rev_corp_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Corp)', 'PRE', 'POST', 'TOL', 'TVS']]

rev_corp_daily_df_display = rev_corp_daily_df.copy()
mod_col_list = rev_corp_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_corp_daily_df_display[col] = rev_corp_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_corp_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Corp),PRE,POST,TOL,TVS
0,202508,20250812,2025-08-13 05:17:31,"-206,654,697",0,0,0,0,0
1,202508,20250811,2025-08-13 05:17:31,"-722,927,923","206,654,697","125,051,358",0,0,0
2,202508,20250810,2025-08-13 05:17:31,"708,329,499","929,582,620","129,981,118","544,214,293","226,512,701","24,802,724"
3,202508,20250809,2025-08-13 05:17:31,"-10,586,822","221,253,121","136,809,620","84,311,401",0,0
4,202508,20250808,2025-08-13 05:17:31,"7,491,828","231,839,943","144,638,078","87,084,163",0,0
5,202508,20250807,2025-08-13 05:17:31,"-468,916,446","224,348,114","139,528,996","84,707,824",0,0
6,202508,20250806,2025-08-13 05:17:31,"457,000,793","693,264,560","144,193,212","390,551,115","148,250,408","7,972,300"
7,202508,20250805,2025-08-13 05:17:31,"-229,706","236,263,767","150,101,109","86,060,161",0,0
8,202508,20250804,2025-08-13 05:17:31,"5,125,434","236,493,473","147,757,069","88,546,795",0,0
9,202508,20250803,2025-08-13 05:17:31,"-1,018,987,320","231,368,039","145,177,178","86,026,858",0,0


## Overview(Geo) by Period

In [9]:
''' Revenue(Geo) Yearly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_yearly_df = chk_src_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_yearly_df = rev_geo_yearly_df.loc[rev_geo_yearly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_yearly_df['TOTAL(Geo)'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B0R000100GEO', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['PRE_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB1R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C'] = np.where(rev_geo_yearly_df['METRIC_CD']=='B2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_T'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['POST_B2C_D'] = np.where(rev_geo_yearly_df['METRIC_CD']=='DB2R010100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TOL'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB3R000100', rev_geo_yearly_df['P'], 0)
rev_geo_yearly_df['TVS'] = np.where(rev_geo_yearly_df['METRIC_CD']=='TB4R000100', rev_geo_yearly_df['P'], 0)

rev_geo_yearly_df = rev_geo_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_yearly_df['YoY'] = rev_geo_yearly_df['TOTAL(Geo)'] - rev_geo_yearly_df['TOTAL(Geo)'].shift()
rev_geo_yearly_df = rev_geo_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
rev_geo_yearly_df = rev_geo_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'YoY', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_yearly_df_display = rev_geo_yearly_df.copy()
rev_geo_yearly_df_display['ACTUAL_AS_OF'] = rev_geo_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_yearly_df_display[col] = rev_geo_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,YoY,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2024,2025-08-13 05:17:31,20241231,0,"142,125,302,342","48,319,930,436","27,043,194,488","21,276,735,948","70,959,878,334","39,407,414,546","31,552,463,789","18,827,249,425","4,018,244,147"
1,2025,2025-08-13 05:17:31,20250811,"-54,719,867,688","87,405,434,654","30,550,947,852","17,606,887,121","12,944,060,731","42,866,619,200","24,489,940,553","18,457,298,559","11,845,278,491","2,061,969,199"


In [10]:
''' Revenue(Geo) Quarterly '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_quarterly_df = chk_src_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].copy()
rev_geo_quarterly_df = rev_geo_quarterly_df.loc[rev_geo_quarterly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_quarterly_df['TOTAL(Geo)'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B0R000100GEO', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['PRE_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB1R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='B2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_T'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['POST_B2C_D'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='DB2R010100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TOL'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB3R000100', rev_geo_quarterly_df['P'], 0)
rev_geo_quarterly_df['TVS'] = np.where(rev_geo_quarterly_df['METRIC_CD']=='TB4R000100', rev_geo_quarterly_df['P'], 0)

rev_geo_quarterly_df = rev_geo_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_quarterly_df['QoQ'] = rev_geo_quarterly_df['TOTAL(Geo)'] - rev_geo_quarterly_df['TOTAL(Geo)'].shift()
rev_geo_quarterly_df = rev_geo_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
rev_geo_quarterly_df = rev_geo_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'QoQ', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_quarterly_df_display = rev_geo_quarterly_df.copy()
rev_geo_quarterly_df_display['ACTUAL_AS_OF'] = rev_geo_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_quarterly_df_display[col] = rev_geo_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,QoQ,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,20241,2025-08-13 05:17:31,20240331,0,"35,361,933,475","11,842,176,786","6,519,610,988","5,322,565,798","17,936,624,018","9,850,451,722","8,086,172,296","4,569,920,004","1,013,212,668"
1,20242,2025-08-13 05:17:31,20240630,"-77,400,943","35,284,532,532","11,858,030,293","6,615,472,616","5,242,557,676","17,792,137,434","9,838,450,875","7,953,686,559","4,665,781,597","968,583,208"
2,20243,2025-08-13 05:17:31,20240930,"422,283,505","35,706,816,037","12,164,985,004","6,830,749,822","5,334,235,182","17,628,033,120","9,834,028,205","7,794,004,915","4,803,785,483","1,110,012,430"
3,20244,2025-08-13 05:17:31,20241231,"65,204,262","35,772,020,298","12,454,738,354","7,077,361,062","5,377,377,292","17,603,083,763","9,884,483,743","7,718,600,020","4,787,762,341","926,435,841"
4,20251,2025-08-13 05:17:31,20250331,"-122,576,840","35,649,443,458","12,375,356,967","7,111,077,784","5,264,279,183","17,580,761,024","9,989,547,936","7,591,213,088","4,810,187,607","883,137,861"
5,20252,2025-08-13 05:17:31,20250630,"-116,422,287","35,533,021,171","12,349,913,007","7,120,338,187","5,229,574,820","17,489,026,867","9,958,948,817","7,530,078,051","4,833,931,536","860,149,762"
6,20253,2025-08-13 05:17:31,20250811,"-19,310,051,146","16,222,970,025","5,825,677,878","3,375,471,150","2,450,206,728","7,796,831,309","4,541,443,800","3,336,007,420","2,201,159,349","318,681,577"


In [11]:
''' Revenue(Geo) Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_monthly_df = chk_src_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr].copy()
rev_geo_monthly_df = rev_geo_monthly_df.loc[rev_geo_monthly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_monthly_df['TOTAL(Geo)'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B0R000100GEO', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['PRE_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB1R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C'] = np.where(rev_geo_monthly_df['METRIC_CD']=='B2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_T'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['POST_B2C_D'] = np.where(rev_geo_monthly_df['METRIC_CD']=='DB2R010100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TOL'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB3R000100', rev_geo_monthly_df['P'], 0)
rev_geo_monthly_df['TVS'] = np.where(rev_geo_monthly_df['METRIC_CD']=='TB4R000100', rev_geo_monthly_df['P'], 0)

rev_geo_monthly_df = rev_geo_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_monthly_df['MoM'] = rev_geo_monthly_df['TOTAL(Geo)'] - rev_geo_monthly_df['TOTAL(Geo)'].shift()
rev_geo_monthly_df = rev_geo_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
rev_geo_monthly_df = rev_geo_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'MoM', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_monthly_df_display = rev_geo_monthly_df.copy()
rev_geo_monthly_df_display['ACTUAL_AS_OF'] = rev_geo_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_monthly_df_display[col] = rev_geo_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,MoM,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-08-13 05:17:31,20240131,0,"11,881,730,399","4,014,674,524","2,211,126,641","1,803,547,883","6,020,820,300","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,20240229,"-255,166,666","11,626,563,733","3,796,713,842","2,082,104,053","1,714,609,789","5,974,961,131","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,20240331,"227,075,609","11,853,639,343","4,030,788,420","2,226,380,294","1,804,408,125","5,940,842,587","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,20240430,"-130,159,939","11,723,479,404","3,883,185,086","2,140,030,223","1,743,154,862","5,961,586,281","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,20240531,"118,388,442","11,841,867,846","4,042,374,933","2,269,268,086","1,773,106,846","5,921,618,672","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,20240630,"-122,682,564","11,719,185,282","3,932,470,274","2,206,174,307","1,726,295,968","5,908,932,481","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,20240731,"138,027,673","11,857,212,955","4,072,664,861","2,272,833,725","1,799,831,136","5,878,727,794","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,20240831,"135,679,548","11,992,892,504","4,109,958,930","2,312,752,271","1,797,206,658","5,880,942,518","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,20240930,"-136,181,926","11,856,710,578","3,982,361,213","2,245,163,825","1,737,197,388","5,868,362,808","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,20241031,"43,673,946","11,900,384,524","4,128,174,548","2,345,529,988","1,782,644,560","5,862,510,509","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [12]:
''' Revenue(Geo) Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_weekly_df = chk_src_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_weekly_df = rev_geo_weekly_df.loc[rev_geo_weekly_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_weekly_df['TOTAL(Geo)'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B0R000100GEO', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['PRE_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB1R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C'] = np.where(rev_geo_weekly_df['METRIC_CD']=='B2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_T'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['POST_B2C_D'] = np.where(rev_geo_weekly_df['METRIC_CD']=='DB2R010100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TOL'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB3R000100', rev_geo_weekly_df['P'], 0)
rev_geo_weekly_df['TVS'] = np.where(rev_geo_weekly_df['METRIC_CD']=='TB4R000100', rev_geo_weekly_df['P'], 0)

rev_geo_weekly_df = rev_geo_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_weekly_df['WoW'] = rev_geo_weekly_df['TOTAL(Geo)'] - rev_geo_weekly_df['TOTAL(Geo)'].shift()
rev_geo_weekly_df = rev_geo_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
rev_geo_weekly_df = rev_geo_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'WoW', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_weekly_df_display = rev_geo_weekly_df.copy()
rev_geo_weekly_df_display['ACTUAL_AS_OF'] = rev_geo_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = rev_geo_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_weekly_df_display[col] = rev_geo_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,WoW,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,2025027,2025-08-13 05:17:31,20250706,0,"2,523,262,811","905,642,039","520,776,631","384,865,408","1,211,899,716","767,819,668","444,080,048","366,697,791","39,023,266"
1,2025028,2025-08-13 05:17:31,20250713,"259,564,101","2,782,826,912","921,096,331","535,899,803","385,196,528","1,413,017,437","833,490,156","579,527,280","403,051,760","45,661,384"
2,2025029,2025-08-13 05:17:31,20250720,"-265,941,513","2,516,885,400","895,004,220","510,657,916","384,346,304","1,249,017,076","711,433,865","537,583,211","334,381,824","38,482,280"
3,2025030,2025-08-13 05:17:31,20250727,"1,034,520","2,517,919,919","900,082,966","512,913,046","387,169,920","1,252,000,322","690,463,921","561,536,401","328,551,447","37,285,184"
4,2025031,2025-08-13 05:17:31,20250803,"448,373,384","2,966,293,304","1,085,791,764","639,830,747","445,961,017","1,361,333,683","819,897,713","541,435,970","393,713,419","125,454,438"
5,2025032,2025-08-13 05:17:31,20250810,"-256,182,895","2,710,110,409","993,009,200","582,071,341","410,937,860","1,309,563,075","718,338,477","591,224,598","374,763,109","32,775,024"
6,2025033,2025-08-13 05:17:31,20250811,"-2,504,439,139","205,671,270","125,051,358","73,321,667","51,729,691",0,0,"80,619,912",0,0


In [13]:
''' Revenue(Geo) Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'B1R000100' #Prepaid Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'B2R010100' #Postpaid Revenue B2C
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

rev_geo_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
rev_geo_daily_df = rev_geo_daily_df.loc[rev_geo_daily_df['METRIC_CD'].isin(v_metric_list)]

rev_geo_daily_df['TOTAL(Geo)'] = np.where(rev_geo_daily_df['METRIC_CD']=='B0R000100GEO', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE'] = np.where(rev_geo_daily_df['METRIC_CD']=='B1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['PRE_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB1R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C'] = np.where(rev_geo_daily_df['METRIC_CD']=='B2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_T'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['POST_B2C_D'] = np.where(rev_geo_daily_df['METRIC_CD']=='DB2R010100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TOL'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB3R000100', rev_geo_daily_df['P'], 0)
rev_geo_daily_df['TVS'] = np.where(rev_geo_daily_df['METRIC_CD']=='TB4R000100', rev_geo_daily_df['P'], 0)

rev_geo_daily_df = rev_geo_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'TOTAL(Geo)':'sum', 'PRE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum'})
rev_geo_daily_df['DoD'] = rev_geo_daily_df['TOTAL(Geo)'] - rev_geo_daily_df['TOTAL(Geo)'].shift()
rev_geo_daily_df = rev_geo_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
rev_geo_daily_df = rev_geo_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'DoD', 'TOTAL(Geo)', 'PRE', 'PRE_T', 'PRE_D', 'POST_B2C', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

rev_geo_daily_df_display = rev_geo_daily_df.copy()
mod_col_list = rev_geo_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    rev_geo_daily_df_display[col] = rev_geo_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
rev_geo_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,DoD,TOTAL(Geo),PRE,PRE_T,PRE_D,POST_B2C,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202508,20250812,2025-08-13 05:17:31,"-205,671,270",0,0,0,0,0,0,0,0,0
1,202508,20250811,2025-08-13 05:17:31,"-682,625,380","205,671,270","125,051,358","73,321,667","51,729,691",0,0,"80,619,912",0,0
2,202508,20250810,2025-08-13 05:17:31,"668,164,483","888,296,650","129,981,118","75,699,345","54,281,773","507,000,107","421,762,492","85,237,615","226,512,701","24,802,724"
3,202508,20250809,2025-08-13 05:17:31,"-7,743,811","220,132,167","136,809,620","80,333,505","56,476,115","83,322,547",0,"83,322,547",0,0
4,202508,20250808,2025-08-13 05:17:31,"4,602,579","227,875,978","144,638,078","85,514,144","59,123,933","83,237,900",0,"83,237,900",0,0
5,202508,20250807,2025-08-13 05:17:31,"-460,228,273","223,273,399","139,528,996","81,929,603","57,599,393","83,744,403",0,"83,744,403",0,0
6,202508,20250806,2025-08-13 05:17:31,"450,532,637","683,501,671","144,193,212","84,088,090","60,105,122","383,085,751","296,575,985","86,509,766","148,250,408","7,972,300"
7,202508,20250805,2025-08-13 05:17:31,"-1,092,476","232,969,034","150,101,109","87,276,316","62,824,793","82,867,925",0,"82,867,925",0,0
8,202508,20250804,2025-08-13 05:17:31,"3,595,271","234,061,510","147,757,069","87,230,338","60,526,731","86,304,441",0,"86,304,441",0,0
9,202508,20250803,2025-08-13 05:17:31,"-747,579,115","230,466,239","145,177,178","83,701,180","61,475,999","85,289,060",0,"85,289,060",0,0


## Products Summary

### Prep Monthly Data

In [14]:
''' Monthly Summary '''

monthly_df = chk_src_df.copy()
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [15]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth
# v_tm_key_mth = 202505

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202507


### All Service

In [16]:
''' All Service (Corp) '''

v_product_grp = 'All Services'

all_service_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_df = all_service_df.loc[~all_service_df['METRIC_CD'].str.contains('GEO')]
all_service_df = all_service_df.loc[all_service_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_df = all_service_df.reset_index(drop=True)
all_service_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,All Services,B0R000100,Total Revenue,2025-08-13 02:18:31,"12,511,966,640","12,490,967,211",0,0,0,"20,999,429"
1,202507,All Services,DB0R000100,Total Revenue : DTAC,2025-08-13 02:18:31,"4,457,188,288","4,457,192,573",0,0,0,"-4,285"
2,202507,All Services,TB0R000100,Total Revenue : TRUE,2025-08-13 02:18:31,"8,054,778,352","8,033,774,638",0,0,0,"21,003,714"


In [17]:
''' All Service (Geo) '''

v_product_grp = 'All Services'

all_service_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp].copy()
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['METRIC_CD'].str.contains('GEO')]
all_service_geo_df = all_service_geo_df.loc[all_service_geo_df['TM_KEY_MTH']==v_tm_key_mth]
all_service_geo_df = all_service_geo_df.reset_index(drop=True)
all_service_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,All Services,B0R000100GEO,Total Revenue (Geo),2025-08-13 05:17:31,0,"11,920,191,869","11,920,038,696","11,919,841,381","11,919,782,342","-11,920,191,869"
1,202507,All Services,DB0R000100GEO,Total Revenue (Geo) : DTAC,2025-08-13 05:17:31,0,"4,268,427,033","4,268,423,580","4,268,360,059","4,268,360,059","-4,268,427,033"
2,202507,All Services,TB0R000100GEO,Total Revenue (Geo) : TRUE,2025-08-13 05:17:31,0,"7,651,764,837","7,651,615,115","7,651,481,322","7,651,422,283","-7,651,764,837"


### Mobile

In [18]:
''' Mobile (Corp) '''

v_product_grp = 'Mobile'

mobile_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_df = mobile_df.loc[~mobile_df['METRIC_CD'].str.contains('GEO')]
mobile_df = mobile_df.loc[mobile_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_df = mobile_df.reset_index(drop=True)
mobile_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Mobile,B0R000101,Mobile Revenue,2025-08-13 02:18:31,"10,632,917,853","10,628,753,825",0,0,0,"4,164,028"
1,202507,Mobile,DB0R000101,Mobile Revenue : DTAC,2025-08-13 02:18:31,"4,457,188,288","4,457,192,573",0,0,0,"-4,285"
2,202507,Mobile,TB0R000101,Mobile Revenue : TMH,2025-08-13 02:18:31,"6,175,729,566","6,171,561,253",0,0,0,"4,168,313"


In [19]:
''' Mobile (Geo) '''

v_product_grp = 'Mobile'

mobile_geo_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['METRIC_CD'].str.contains('GEO')]
mobile_geo_df = mobile_geo_df.loc[mobile_geo_df['TM_KEY_MTH']==v_tm_key_mth]
mobile_geo_df = mobile_geo_df.reset_index(drop=True)
mobile_geo_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Mobile,B0R000101GEO,Mobile Revenue (Geo),2025-08-13 05:17:31,0,"10,057,978,484","10,057,825,310","10,057,748,333","10,057,748,333","-10,057,978,484"
1,202507,Mobile,DB0R000101GEO,Mobile Revenue (Geo) : DTAC,2025-08-13 05:17:31,0,"4,268,427,033","4,268,423,580","4,268,360,059","4,268,360,059","-4,268,427,033"
2,202507,Mobile,TB0R000101GEO,Mobile Revenue (Geo) : TMH,2025-08-13 05:17:31,0,"5,789,551,451","5,789,401,730","5,789,388,274","5,789,388,274","-5,789,551,451"


### Prepaid

In [20]:
''' Prepaid '''

v_product_grp = 'Prepaid'

prepaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
prepaid_df = prepaid_df.loc[prepaid_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_df = prepaid_df.reset_index(drop=True)
prepaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Prepaid,B1R000100,Prepaid Revenue,2025-08-13 05:17:31,"4,232,291,456","4,232,291,456","4,232,153,471","4,232,087,268","4,232,087,268",0
1,202507,Prepaid,DB1R000100,Prepaid Revenue : DTAC,2025-08-13 05:17:31,"1,788,083,008","1,788,083,008","1,788,083,008","1,788,029,861","1,788,029,861",0
2,202507,Prepaid,TB1R000100,Prepaid Revenue : TMH,2025-08-13 05:17:31,"2,444,208,447","2,444,208,447","2,444,070,463","2,444,057,407","2,444,057,407",0


### Postpaid

In [21]:
''' Postpaid '''

v_product_grp = 'Postpaid'

postpaid_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_df = postpaid_df.loc[~postpaid_df['METRIC_NAME'].str.contains('B2C|B2B')]
postpaid_df = postpaid_df.loc[postpaid_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_df = postpaid_df.reset_index(drop=True)
postpaid_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Postpaid,B2R000100,Postpaid Revenue,2025-08-13 02:18:31,"6,396,462,370","6,396,462,370",0,0,0,0
1,202507,Postpaid,DB2R000100,Postpaid Revenue : DTAC,2025-08-13 05:17:31,"2,669,109,565","2,669,109,565","2,669,106,112","2,667,980,276","2,667,980,276",0
2,202507,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,727,352,805","3,727,352,805",0,0,0,0


In [22]:
''' Postpaid B2C '''

v_product_grp = 'Postpaid'

postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['METRIC_NAME'].str.contains('B2C')]
postpaid_b2c_df = postpaid_b2c_df.loc[postpaid_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2c_df = postpaid_b2c_df.reset_index(drop=True)
postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Postpaid,B2R010100,Postpaid Revenue B2C,2025-08-13 05:17:31,"5,825,687,028","5,825,687,028","5,825,671,839","5,825,661,065","5,825,661,065",0
1,202507,Postpaid,DB2R010100,Postpaid Revenue B2C : DTAC,2025-08-13 05:17:31,"2,480,344,024","2,480,344,024","2,480,340,572","2,480,330,197","2,480,330,197",0
2,202507,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,345,343,003","3,345,343,003","3,345,331,267","3,345,330,868","3,345,330,868",0


In [23]:
''' Postpaid B2B '''

v_product_grp = 'Postpaid'

postpaid_b2b_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['METRIC_NAME'].str.contains('B2B')]
postpaid_b2b_df = postpaid_b2b_df.loc[postpaid_b2b_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_b2b_df = postpaid_b2b_df.reset_index(drop=True)
postpaid_b2b_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,Postpaid,B2R020100,Postpaid Revenue B2B,2025-08-13 02:18:31,"570,774,143","570,774,143",0,0,0,0
1,202507,Postpaid,DB2R020100,Postpaid Revenue B2B : DTAC,2025-08-13 05:17:31,"188,765,540","188,765,540","188,765,540","187,650,078","187,650,078",0
2,202507,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"382,008,603","382,008,603",0,0,0,0


### TOL

In [24]:
''' TOL '''

v_product_grp = 'TOL'

tol_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tol_df = tol_df.loc[tol_df['TM_KEY_MTH']==v_tm_key_mth]
tol_df = tol_df.reset_index(drop=True)
tol_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,TOL,TB3R000100,TOL Revenue,2025-08-13 05:17:31,"1,605,399,456","1,605,399,456","1,605,399,456","1,605,399,456","1,605,399,456",0


### TVS

In [25]:
''' TVS '''

v_product_grp = 'TVS'

tvs_df = monthly_df_display.loc[monthly_df_display['PRODUCT_GRP']==v_product_grp]
tvs_df = tvs_df.loc[tvs_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_df = tvs_df.reset_index(drop=True)
tvs_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202507,TVS,TB4R000100,TVS Revenue,2025-08-13 05:17:31,"256,813,929","256,813,929","256,813,929","256,693,591","256,634,553",0


## ** Special Metric (Corp & Geo)

### Revenue Corporate (C)
    B0R000100	Total Revenue
    DB0R000100	Total Revenue : DTAC
    TB0R000100	Total Revenue : TRUE
    
    B0R000101	Mobile Revenue
    DB0R000101	Mobile Revenue : DTAC
    TB0R000101	Mobile Revenue : TMH

In [26]:
''' B0R000100 : Total Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'TB1R000100', 'DB2R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100', 'TNSC00147')

v_metric_list = [
    'B0R000100' #Total Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_df = total_rev_corp_df.loc[total_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_df['TOTAL'] = np.where(total_rev_corp_df['METRIC_CD']=='B0R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['PRE_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB1R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_T'] = np.where(total_rev_corp_df['METRIC_CD']=='TB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['POST_D'] = np.where(total_rev_corp_df['METRIC_CD']=='DB2R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TOL'] = np.where(total_rev_corp_df['METRIC_CD']=='TB3R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['TVS'] = np.where(total_rev_corp_df['METRIC_CD']=='TB4R000100', total_rev_corp_df['C'], 0)
total_rev_corp_df['CHK_SUM'] = np.where(total_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100','TB3R000100','TB4R000100']), total_rev_corp_df['C'], 0)

total_rev_corp_df = total_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_df['CHK_DIFF'] = total_rev_corp_df['TOTAL'] - total_rev_corp_df['CHK_SUM']
total_rev_corp_df = total_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_df = total_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_df[col] = total_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_T,POST_D,TOL,TVS
0,202401,2025-08-13 05:17:31,"33,068,240","12,494,707,100","12,527,775,340","2,211,126,641","1,803,547,883","3,722,556,455","2,911,240,546","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,"23,506,585","12,224,536,694","12,248,043,279","2,082,104,053","1,714,609,789","3,686,285,226","2,886,648,865","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,"25,129,495","12,459,268,326","12,484,397,821","2,226,380,294","1,804,408,125","3,658,095,837","2,888,375,734","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,"23,870,945","12,334,721,853","12,358,592,798","2,140,030,223","1,743,154,862","3,692,736,895","2,880,091,836","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,"25,540,442","12,440,265,912","12,465,806,354","2,269,268,086","1,773,106,846","3,675,457,906","2,844,558,832","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,"23,747,283","12,347,976,614","12,371,723,897","2,206,174,307","1,726,295,968","3,712,964,588","2,824,759,226","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,"26,564,154","12,463,315,685","12,489,879,839","2,272,833,725","1,799,831,136","3,667,787,168","2,817,043,356","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,"29,588,466","12,594,999,452","12,624,587,918","2,312,752,271","1,797,206,658","3,680,722,062","2,802,327,404","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,"30,443,177","12,479,579,804","12,510,022,981","2,245,163,825","1,737,197,388","3,706,292,256","2,784,939,778","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,"45,904,631","12,523,605,918","12,569,510,549","2,345,529,988","1,782,644,560","3,685,237,458","2,800,494,445","1,598,256,224","311,443,243"


In [27]:
''' TB0R000100 : Total Revenue : TRUE '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB1R000100', 'TB2R000100', 'TB3R000100', 'TB4R000100')

v_metric_list = [
    'TB0R000100' #Total Revenue : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_corp_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_true_df = total_rev_corp_true_df.loc[total_rev_corp_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_true_df['TOTAL_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB0R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['PRE_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB1R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['POST_T'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB2R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TOL'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB3R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['TVS'] = np.where(total_rev_corp_true_df['METRIC_CD']=='TB4R000100', total_rev_corp_true_df['C'], 0)
total_rev_corp_true_df['CHK_SUM'] = np.where(total_rev_corp_true_df['METRIC_CD'].isin(['TB1R000100','TB2R000100','TB3R000100','TB4R000100']), total_rev_corp_true_df['C'], 0)

total_rev_corp_true_df = total_rev_corp_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_corp_true_df['CHK_DIFF'] = total_rev_corp_true_df['TOTAL_T'] - total_rev_corp_true_df['CHK_SUM']
total_rev_corp_true_df = total_rev_corp_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_true_df = total_rev_corp_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_T', 'TOL', 'TVS']]

mod_col_list = total_rev_corp_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_true_df[col] = total_rev_corp_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_T,TOL,TVS
0,202401,2025-08-13 05:17:31,"33,180,744","7,779,918,671","7,813,099,416","2,211,126,641","3,722,556,455","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,"23,617,016","7,623,278,040","7,646,895,055","2,082,104,053","3,686,285,226","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,"25,216,744","7,766,484,467","7,791,701,211","2,226,380,294","3,658,095,837","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,"23,962,132","7,711,475,155","7,735,437,287","2,140,030,223","3,692,736,895","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,"25,620,601","7,822,600,234","7,848,220,835","2,269,268,086","3,675,457,906","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,"23,832,682","7,796,921,421","7,820,754,103","2,206,174,307","3,712,964,588","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,"26,666,256","7,846,441,194","7,873,107,450","2,272,833,725","3,667,787,168","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,"29,693,737","7,995,465,389","8,025,159,126","2,312,752,271","3,680,722,062","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,"30,526,907","7,957,442,638","7,987,969,545","2,245,163,825","3,706,292,256","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,"45,999,504","7,940,466,913","7,986,466,416","2,345,529,988","3,685,237,458","1,598,256,224","311,443,243"


In [28]:
''' DB0R000100 : Total Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB1R000100', 'DB2R000100')

v_metric_list = [
    'DB0R000100' #Total Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

total_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
total_rev_corp_dtac_df = total_rev_corp_dtac_df.loc[total_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_corp_dtac_df['TOTAL_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB0R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['PRE_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['POST_D'] = np.where(total_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', total_rev_corp_dtac_df['C'], 0)
total_rev_corp_dtac_df['CHK_SUM'] = np.where(total_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), total_rev_corp_dtac_df['C'], 0)

total_rev_corp_dtac_df = total_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
total_rev_corp_dtac_df['CHK_DIFF'] = total_rev_corp_dtac_df['TOTAL_D'] - total_rev_corp_dtac_df['CHK_SUM']
total_rev_corp_dtac_df = total_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_corp_dtac_df = total_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_D']]

mod_col_list = total_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_corp_dtac_df[col] = total_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_D
0,202401,2025-08-13 05:17:31,"-112,505","4,714,788,429","4,714,675,924","1,803,547,883","2,911,240,546"
1,202402,2025-08-13 05:17:31,"-110,430","4,601,258,654","4,601,148,224","1,714,609,789","2,886,648,865"
2,202403,2025-08-13 05:17:31,"-87,249","4,692,783,860","4,692,696,611","1,804,408,125","2,888,375,734"
3,202404,2025-08-13 05:17:31,"-91,187","4,623,246,698","4,623,155,511","1,743,154,862","2,880,091,836"
4,202405,2025-08-13 05:17:31,"-80,159","4,617,665,679","4,617,585,519","1,773,106,846","2,844,558,832"
5,202406,2025-08-13 05:17:31,"-85,399","4,551,055,193","4,550,969,794","1,726,295,968","2,824,759,226"
6,202407,2025-08-13 05:17:31,"-102,102","4,616,874,491","4,616,772,389","1,799,831,136","2,817,043,356"
7,202408,2025-08-13 05:17:31,"-105,271","4,599,534,063","4,599,428,792","1,797,206,658","2,802,327,404"
8,202409,2025-08-13 05:17:31,"-83,730","4,522,137,166","4,522,053,436","1,737,197,388","2,784,939,778"
9,202410,2025-08-13 05:17:31,"-94,873","4,583,139,005","4,583,044,133","1,782,644,560","2,800,494,445"


In [29]:
''' B0R000101 : Mobile Revenue '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'TB2R000100', 'DB1R000100', 'TB1R000100')

v_metric_list = [
    'B0R000101' #Mobile Revenue
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R000100' #Postpaid Revenue : DTAC
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_df = mobile_rev_corp_df.loc[mobile_rev_corp_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_df['MOBILE'] = np.where(mobile_rev_corp_df['METRIC_CD']=='B0R000101', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['PRE_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_T'] = np.where(mobile_rev_corp_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['POST_D'] = np.where(mobile_rev_corp_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_df['C'], 0)
mobile_rev_corp_df['CHK_SUM'] = np.where(mobile_rev_corp_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R000100','DB2R000100']), mobile_rev_corp_df['C'], 0)

mobile_rev_corp_df = mobile_rev_corp_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_T':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_df['CHK_DIFF'] = mobile_rev_corp_df['MOBILE'] - mobile_rev_corp_df['CHK_SUM']
mobile_rev_corp_df = mobile_rev_corp_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_df = mobile_rev_corp_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_T', 'POST_D']]

mod_col_list = mobile_rev_corp_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_df[col] = mobile_rev_corp_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_T,POST_D
0,202401,2025-08-13 05:17:31,"88,428","10,648,471,525","10,648,559,952","2,211,126,641","1,803,547,883","3,722,556,455","2,911,240,546"
1,202402,2025-08-13 05:17:31,"273,318","10,369,647,933","10,369,921,251","2,082,104,053","1,714,609,789","3,686,285,226","2,886,648,865"
2,202403,2025-08-13 05:17:31,"555,550","10,577,259,991","10,577,815,541","2,226,380,294","1,804,408,125","3,658,095,837","2,888,375,734"
3,202404,2025-08-13 05:17:31,"1,184,927","10,456,013,816","10,457,198,744","2,140,030,223","1,743,154,862","3,692,736,895","2,880,091,836"
4,202405,2025-08-13 05:17:31,"4,094,170","10,562,391,671","10,566,485,841","2,269,268,086","1,773,106,846","3,675,457,906","2,844,558,832"
5,202406,2025-08-13 05:17:31,"4,013,650","10,470,194,088","10,474,207,738","2,206,174,307","1,726,295,968","3,712,964,588","2,824,759,226"
6,202407,2025-08-13 05:17:31,"4,676,194","10,557,495,385","10,562,171,579","2,272,833,725","1,799,831,136","3,667,787,168","2,817,043,356"
7,202408,2025-08-13 05:17:31,"4,177,481","10,593,008,396","10,597,185,877","2,312,752,271","1,797,206,658","3,680,722,062","2,802,327,404"
8,202409,2025-08-13 05:17:31,"2,880,349","10,473,593,247","10,476,473,596","2,245,163,825","1,737,197,388","3,706,292,256","2,784,939,778"
9,202410,2025-08-13 05:17:31,"3,753,058","10,613,906,451","10,617,659,509","2,345,529,988","1,782,644,560","3,685,237,458","2,800,494,445"


In [30]:
''' TB0R000101 : Mobile Revenue : TMH '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('TB2R000100', 'TB1R000100')

v_metric_list = [
    'TB0R000101' #Mobile Revenue : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R000100' #Postpaid Revenue : TMH
    ]

mobile_rev_corp_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.loc[mobile_rev_corp_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_tmh_df['MOBILE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB0R000101', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['PRE_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['POST_T'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD']=='TB2R000100', mobile_rev_corp_tmh_df['C'], 0)
mobile_rev_corp_tmh_df['CHK_SUM'] = np.where(mobile_rev_corp_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R000100']), mobile_rev_corp_tmh_df['C'], 0)

mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_tmh_df['CHK_DIFF'] = mobile_rev_corp_tmh_df['MOBILE_T'] - mobile_rev_corp_tmh_df['CHK_SUM']
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_tmh_df = mobile_rev_corp_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_T']]

mod_col_list = mobile_rev_corp_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_tmh_df[col] = mobile_rev_corp_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_T
0,202401,2025-08-13 05:17:31,"200,932","5,933,683,096","5,933,884,028","2,211,126,641","3,722,556,455"
1,202402,2025-08-13 05:17:31,"383,748","5,768,389,279","5,768,773,028","2,082,104,053","3,686,285,226"
2,202403,2025-08-13 05:17:31,"642,799","5,884,476,131","5,885,118,930","2,226,380,294","3,658,095,837"
3,202404,2025-08-13 05:17:31,"1,276,114","5,832,767,118","5,834,043,232","2,140,030,223","3,692,736,895"
4,202405,2025-08-13 05:17:31,"4,174,329","5,944,725,992","5,948,900,322","2,269,268,086","3,675,457,906"
5,202406,2025-08-13 05:17:31,"4,099,049","5,919,138,895","5,923,237,944","2,206,174,307","3,712,964,588"
6,202407,2025-08-13 05:17:31,"4,778,296","5,940,620,894","5,945,399,190","2,272,833,725","3,667,787,168"
7,202408,2025-08-13 05:17:31,"4,282,752","5,993,474,334","5,997,757,085","2,312,752,271","3,680,722,062"
8,202409,2025-08-13 05:17:31,"2,964,079","5,951,456,081","5,954,420,160","2,245,163,825","3,706,292,256"
9,202410,2025-08-13 05:17:31,"3,847,930","6,030,767,446","6,034,615,376","2,345,529,988","3,685,237,458"


In [31]:
''' DB0R000101 : Mobile Revenue : DTAC '''

# Create KPI(All area both actual & target) by sum METRIC_CD :
# ('DB2R000100', 'DB1R000100')

v_metric_list = [
    'DB0R000101' #Mobile Revenue : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R000100' #Postpaid Revenue : DTAC
    ]

mobile_rev_corp_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'C']].copy()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.loc[mobile_rev_corp_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_corp_dtac_df['MOBILE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB0R000101', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['PRE_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['POST_D'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD']=='DB2R000100', mobile_rev_corp_dtac_df['C'], 0)
mobile_rev_corp_dtac_df['CHK_SUM'] = np.where(mobile_rev_corp_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R000100']), mobile_rev_corp_dtac_df['C'], 0)

mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_corp_dtac_df['CHK_DIFF'] = mobile_rev_corp_dtac_df['MOBILE_D'] - mobile_rev_corp_dtac_df['CHK_SUM']
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_corp_dtac_df = mobile_rev_corp_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_D']]

mod_col_list = mobile_rev_corp_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_corp_dtac_df[col] = mobile_rev_corp_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_corp_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_D
0,202401,2025-08-13 05:17:31,"-112,505","4,714,788,429","4,714,675,924","1,803,547,883","2,911,240,546"
1,202402,2025-08-13 05:17:31,"-110,430","4,601,258,654","4,601,148,224","1,714,609,789","2,886,648,865"
2,202403,2025-08-13 05:17:31,"-87,249","4,692,783,860","4,692,696,611","1,804,408,125","2,888,375,734"
3,202404,2025-08-13 05:17:31,"-91,187","4,623,246,698","4,623,155,511","1,743,154,862","2,880,091,836"
4,202405,2025-08-13 05:17:31,"-80,159","4,617,665,679","4,617,585,519","1,773,106,846","2,844,558,832"
5,202406,2025-08-13 05:17:31,"-85,399","4,551,055,193","4,550,969,794","1,726,295,968","2,824,759,226"
6,202407,2025-08-13 05:17:31,"-102,102","4,616,874,491","4,616,772,389","1,799,831,136","2,817,043,356"
7,202408,2025-08-13 05:17:31,"-105,271","4,599,534,063","4,599,428,792","1,797,206,658","2,802,327,404"
8,202409,2025-08-13 05:17:31,"-83,730","4,522,137,166","4,522,053,436","1,737,197,388","2,784,939,778"
9,202410,2025-08-13 05:17:31,"-94,873","4,583,139,005","4,583,044,133","1,782,644,560","2,800,494,445"


### Revenue Geo (P)
    B0R000100GEO	Total Revenue (Geo)
    DB0R000100GEO	Total Revenue (Geo) : DTAC
    TB0R000100GEO	Total Revenue (Geo) : TRUE
    
    B0R000101GEO	Mobile Revenue (Geo)
    DB0R000101GEO	Mobile Revenue (Geo) : DTAC
    TB0R000101GEO	Mobile Revenue (Geo) : TMH

In [32]:
''' B0R000100GEO : Total Revenue (Geo) '''

v_metric_list = [
    'B0R000100GEO' #Total Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_df = total_rev_geo_df.loc[total_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_df['TOTAL'] = np.where(total_rev_geo_df['METRIC_CD']=='B0R000100GEO', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['PRE_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB1R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_T'] = np.where(total_rev_geo_df['METRIC_CD']=='TB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['POST_B2C_D'] = np.where(total_rev_geo_df['METRIC_CD']=='DB2R010100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TOL'] = np.where(total_rev_geo_df['METRIC_CD']=='TB3R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['TVS'] = np.where(total_rev_geo_df['METRIC_CD']=='TB4R000100', total_rev_geo_df['P'], 0)
total_rev_geo_df['CHK_SUM'] = np.where(total_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100','TB3R000100','TB4R000100']), total_rev_geo_df['P'], 0)

total_rev_geo_df = total_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_df['CHK_DIFF'] = total_rev_geo_df['TOTAL'] - total_rev_geo_df['CHK_SUM']
total_rev_geo_df = total_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_df = total_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_df[col] = total_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D,TOL,TVS
0,202401,2025-08-13 05:17:31,0,"11,881,730,399","11,881,730,399","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,0,"11,626,563,733","11,626,563,733","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,0,"11,853,639,343","11,853,639,343","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,0,"11,723,479,404","11,723,479,404","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,0,"11,841,867,846","11,841,867,846","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,0,"11,719,185,282","11,719,185,282","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,0,"11,857,212,955","11,857,212,955","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,-0,"11,992,892,504","11,992,892,504","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,0,"11,856,710,578","11,856,710,578","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,0,"11,900,384,524","11,900,384,524","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955","1,598,256,224","311,443,243"


In [33]:
''' TB0R000100GEO : Total Revenue (Geo) : TRUE '''

v_metric_list = [
    'TB0R000100GEO' #Total Revenue (Geo) : TRUE
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    , 'TB3R000100' #TOL Revenue
    , 'TB4R000100' #TVS Revenue
    ]

total_rev_geo_true_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_true_df = total_rev_geo_true_df.loc[total_rev_geo_true_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_true_df['TOTAL_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB0R000100GEO', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['PRE_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB1R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['POST_B2C_T'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB2R010100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TOL'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB3R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['TVS'] = np.where(total_rev_geo_true_df['METRIC_CD']=='TB4R000100', total_rev_geo_true_df['P'], 0)
total_rev_geo_true_df['CHK_SUM'] = np.where(total_rev_geo_true_df['METRIC_CD'].isin(['TB1R000100','TB2R010100','TB3R000100','TB4R000100']), total_rev_geo_true_df['P'], 0)

total_rev_geo_true_df = total_rev_geo_true_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'TOL':'sum', 'TVS':'sum', 'CHK_SUM':'sum'})
total_rev_geo_true_df['CHK_DIFF'] = total_rev_geo_true_df['TOTAL_T'] - total_rev_geo_true_df['CHK_SUM']
total_rev_geo_true_df = total_rev_geo_true_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_true_df = total_rev_geo_true_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_T', 'PRE_T', 'POST_B2C_T', 'TOL', 'TVS']]

mod_col_list = total_rev_geo_true_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_true_df[col] = total_rev_geo_true_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_true_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_T,PRE_T,POST_B2C_T,TOL,TVS
0,202401,2025-08-13 05:17:31,0,"7,368,644,248","7,368,644,248","2,211,126,641","3,311,282,032","1,507,069,397","339,166,179"
1,202402,2025-08-13 05:17:31,0,"7,222,065,281","7,222,065,281","2,082,104,053","3,285,072,468","1,523,039,176","331,849,584"
2,202403,2025-08-13 05:17:31,0,"7,362,485,852","7,362,485,852","2,226,380,294","3,254,097,222","1,539,811,431","342,196,905"
3,202404,2025-08-13 05:17:31,0,"7,298,917,238","7,298,917,238","2,140,030,223","3,280,178,978","1,546,917,278","331,790,759"
4,202405,2025-08-13 05:17:31,0,"7,420,456,954","7,420,456,954","2,269,268,086","3,273,314,626","1,554,971,583","322,902,658"
5,202406,2025-08-13 05:17:31,0,"7,368,914,105","7,368,914,105","2,206,174,307","3,284,957,272","1,563,892,736","313,889,791"
6,202407,2025-08-13 05:17:31,0,"7,447,886,733","7,447,886,733","2,272,833,725","3,269,232,707","1,577,812,706","328,007,594"
7,202408,2025-08-13 05:17:31,0,"7,598,058,116","7,598,058,116","2,312,752,271","3,283,314,789","1,592,482,287","409,508,769"
8,202409,2025-08-13 05:17:31,0,"7,532,631,092","7,532,631,092","2,245,163,825","3,281,480,710","1,633,490,490","372,496,067"
9,202410,2025-08-13 05:17:31,0,"7,527,416,008","7,527,416,008","2,345,529,988","3,272,186,553","1,598,256,224","311,443,243"


In [34]:
''' DB0R000100GEO : Total Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000100GEO' #Total Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

total_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
total_rev_geo_dtac_df = total_rev_geo_dtac_df.loc[total_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

total_rev_geo_dtac_df['TOTAL_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB0R000100GEO', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['PRE_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['POST_B2C_D'] = np.where(total_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', total_rev_geo_dtac_df['P'], 0)
total_rev_geo_dtac_df['CHK_SUM'] = np.where(total_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), total_rev_geo_dtac_df['P'], 0)

total_rev_geo_dtac_df = total_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'TOTAL_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
total_rev_geo_dtac_df['CHK_DIFF'] = total_rev_geo_dtac_df['TOTAL_D'] - total_rev_geo_dtac_df['CHK_SUM']
total_rev_geo_dtac_df = total_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
total_rev_geo_dtac_df = total_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'TOTAL_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = total_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    total_rev_geo_dtac_df[col] = total_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
total_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,TOTAL_D,PRE_D,POST_B2C_D
0,202401,2025-08-13 05:17:31,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-08-13 05:17:31,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-08-13 05:17:31,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-08-13 05:17:31,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-08-13 05:17:31,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-08-13 05:17:31,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-08-13 05:17:31,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-08-13 05:17:31,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-08-13 05:17:31,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-08-13 05:17:31,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


In [35]:
''' B0R000101GEO : Mobile Revenue (Geo) '''

v_metric_list = [
    'B0R000101GEO' #Mobile Revenue (Geo)
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_df = mobile_rev_geo_df.loc[mobile_rev_geo_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_df['MOBILE'] = np.where(mobile_rev_geo_df['METRIC_CD']=='B0R000101GEO', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['PRE_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_T'] = np.where(mobile_rev_geo_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['POST_B2C_D'] = np.where(mobile_rev_geo_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_df['P'], 0)
mobile_rev_geo_df['CHK_SUM'] = np.where(mobile_rev_geo_df['METRIC_CD'].isin(['TB1R000100','DB1R000100','TB2R010100','DB2R010100']), mobile_rev_geo_df['P'], 0)

mobile_rev_geo_df = mobile_rev_geo_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE':'sum', 'PRE_T':'sum', 'PRE_D':'sum', 'POST_B2C_T':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_df['CHK_DIFF'] = mobile_rev_geo_df['MOBILE'] - mobile_rev_geo_df['CHK_SUM']
mobile_rev_geo_df = mobile_rev_geo_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_df = mobile_rev_geo_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE', 'PRE_T', 'PRE_D', 'POST_B2C_T', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_df[col] = mobile_rev_geo_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE,PRE_T,PRE_D,POST_B2C_T,POST_B2C_D
0,202401,2025-08-13 05:17:31,0,"10,035,494,824","10,035,494,824","2,211,126,641","1,803,547,883","3,311,282,032","2,709,538,268"
1,202402,2025-08-13 05:17:31,0,"9,771,674,973","9,771,674,973","2,082,104,053","1,714,609,789","3,285,072,468","2,689,888,663"
2,202403,2025-08-13 05:17:31,0,"9,971,631,007","9,971,631,007","2,226,380,294","1,804,408,125","3,254,097,222","2,686,745,365"
3,202404,2025-08-13 05:17:31,0,"9,844,771,366","9,844,771,366","2,140,030,223","1,743,154,862","3,280,178,978","2,681,407,303"
4,202405,2025-08-13 05:17:31,-0,"9,963,993,605","9,963,993,605","2,269,268,086","1,773,106,846","3,273,314,626","2,648,304,046"
5,202406,2025-08-13 05:17:31,0,"9,841,402,756","9,841,402,756","2,206,174,307","1,726,295,968","3,284,957,272","2,623,975,210"
6,202407,2025-08-13 05:17:31,0,"9,951,392,655","9,951,392,655","2,272,833,725","1,799,831,136","3,269,232,707","2,609,495,087"
7,202408,2025-08-13 05:17:31,0,"9,990,901,448","9,990,901,448","2,312,752,271","1,797,206,658","3,283,314,789","2,597,627,729"
8,202409,2025-08-13 05:17:31,0,"9,850,724,021","9,850,724,021","2,245,163,825","1,737,197,388","3,281,480,710","2,586,882,098"
9,202410,2025-08-13 05:17:31,0,"9,990,685,057","9,990,685,057","2,345,529,988","1,782,644,560","3,272,186,553","2,590,323,955"


In [36]:
''' TB0R000101GEO : Mobile Revenue (Geo) : TMH '''

v_metric_list = [
    'TB0R000101GEO' #Mobile Revenue (Geo) : TMH
    , 'TB1R000100' #Prepaid Revenue : TMH
    , 'TB2R010100' #Postpaid Revenue B2C : TMH
    ]

mobile_rev_geo_tmh_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.loc[mobile_rev_geo_tmh_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_tmh_df['MOBILE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB0R000101GEO', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['PRE_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB1R000100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['POST_B2C_T'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD']=='TB2R010100', mobile_rev_geo_tmh_df['P'], 0)
mobile_rev_geo_tmh_df['CHK_SUM'] = np.where(mobile_rev_geo_tmh_df['METRIC_CD'].isin(['TB1R000100','TB2R010100']), mobile_rev_geo_tmh_df['P'], 0)

mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_T':'sum', 'PRE_T':'sum', 'POST_B2C_T':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_tmh_df['CHK_DIFF'] = mobile_rev_geo_tmh_df['MOBILE_T'] - mobile_rev_geo_tmh_df['CHK_SUM']
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_tmh_df = mobile_rev_geo_tmh_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_T', 'PRE_T', 'POST_B2C_T']]

mod_col_list = mobile_rev_geo_tmh_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_tmh_df[col] = mobile_rev_geo_tmh_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_tmh_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_T,PRE_T,POST_B2C_T
0,202401,2025-08-13 05:17:31,0,"5,522,408,673","5,522,408,673","2,211,126,641","3,311,282,032"
1,202402,2025-08-13 05:17:31,-0,"5,367,176,521","5,367,176,521","2,082,104,053","3,285,072,468"
2,202403,2025-08-13 05:17:31,0,"5,480,477,516","5,480,477,516","2,226,380,294","3,254,097,222"
3,202404,2025-08-13 05:17:31,0,"5,420,209,201","5,420,209,201","2,140,030,223","3,280,178,978"
4,202405,2025-08-13 05:17:31,-0,"5,542,582,712","5,542,582,712","2,269,268,086","3,273,314,626"
5,202406,2025-08-13 05:17:31,0,"5,491,131,578","5,491,131,578","2,206,174,307","3,284,957,272"
6,202407,2025-08-13 05:17:31,0,"5,542,066,432","5,542,066,432","2,272,833,725","3,269,232,707"
7,202408,2025-08-13 05:17:31,0,"5,596,067,060","5,596,067,060","2,312,752,271","3,283,314,789"
8,202409,2025-08-13 05:17:31,0,"5,526,644,535","5,526,644,535","2,245,163,825","3,281,480,710"
9,202410,2025-08-13 05:17:31,0,"5,617,716,541","5,617,716,541","2,345,529,988","3,272,186,553"


In [37]:
''' DB0R000101GEO : Mobile Revenue (Geo) : DTAC '''

v_metric_list = [
    'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
    , 'DB1R000100' #Prepaid Revenue : DTAC
    , 'DB2R010100' #Postpaid Revenue B2C : DTAC
    ]

mobile_rev_geo_dtac_df = monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.loc[mobile_rev_geo_dtac_df['METRIC_CD'].isin(v_metric_list)]

mobile_rev_geo_dtac_df['MOBILE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB0R000101GEO', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['PRE_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB1R000100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['POST_B2C_D'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD']=='DB2R010100', mobile_rev_geo_dtac_df['P'], 0)
mobile_rev_geo_dtac_df['CHK_SUM'] = np.where(mobile_rev_geo_dtac_df['METRIC_CD'].isin(['DB1R000100','DB2R010100']), mobile_rev_geo_dtac_df['P'], 0)

mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.groupby(['TM_KEY_MTH']).agg({'PPN_TM':'max', 'MOBILE_D':'sum', 'PRE_D':'sum', 'POST_B2C_D':'sum', 'CHK_SUM':'sum'})
mobile_rev_geo_dtac_df['CHK_DIFF'] = mobile_rev_geo_dtac_df['MOBILE_D'] - mobile_rev_geo_dtac_df['CHK_SUM']
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
mobile_rev_geo_dtac_df = mobile_rev_geo_dtac_df[['TM_KEY_MTH', 'PPN_TM', 'CHK_DIFF', 'CHK_SUM', 'MOBILE_D', 'PRE_D', 'POST_B2C_D']]

mod_col_list = mobile_rev_geo_dtac_df.iloc[:, 2:].columns.tolist()
for col in mod_col_list:
    mobile_rev_geo_dtac_df[col] = mobile_rev_geo_dtac_df[col].apply(lambda x: format(x, ',.0f'))
mobile_rev_geo_dtac_df

,TM_KEY_MTH,PPN_TM,CHK_DIFF,CHK_SUM,MOBILE_D,PRE_D,POST_B2C_D
0,202401,2025-08-13 05:17:31,0,"4,513,086,151","4,513,086,151","1,803,547,883","2,709,538,268"
1,202402,2025-08-13 05:17:31,0,"4,404,498,452","4,404,498,452","1,714,609,789","2,689,888,663"
2,202403,2025-08-13 05:17:31,0,"4,491,153,491","4,491,153,491","1,804,408,125","2,686,745,365"
3,202404,2025-08-13 05:17:31,0,"4,424,562,165","4,424,562,165","1,743,154,862","2,681,407,303"
4,202405,2025-08-13 05:17:31,0,"4,421,410,892","4,421,410,892","1,773,106,846","2,648,304,046"
5,202406,2025-08-13 05:17:31,0,"4,350,271,177","4,350,271,177","1,726,295,968","2,623,975,210"
6,202407,2025-08-13 05:17:31,0,"4,409,326,223","4,409,326,223","1,799,831,136","2,609,495,087"
7,202408,2025-08-13 05:17:31,0,"4,394,834,388","4,394,834,388","1,797,206,658","2,597,627,729"
8,202409,2025-08-13 05:17:31,-0,"4,324,079,486","4,324,079,486","1,737,197,388","2,586,882,098"
9,202410,2025-08-13 05:17:31,0,"4,372,968,515","4,372,968,515","1,782,644,560","2,590,323,955"


## ** Issue History

In [38]:
''' Issue : Postpaid Revenue : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R000100'

issue_postpaid_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.loc[issue_postpaid_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_df = issue_postpaid_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,722,556,455","3,722,556,455",0,0,0,0
1,202402,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,686,285,226","3,686,285,226",0,0,0,0
2,202403,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,658,095,837","3,658,095,837",0,0,0,0
3,202404,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,692,736,895","3,692,736,895",0,0,0,0
4,202405,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,675,457,906","3,675,457,906",0,0,0,0
5,202406,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,712,964,588","3,712,964,588",0,0,0,0
6,202407,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,667,787,168","3,667,787,168",0,0,0,0
7,202408,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,680,722,062","3,680,722,062",0,0,0,0
8,202409,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,706,292,256","3,706,292,256",0,0,0,0
9,202410,Postpaid,TB2R000100,Postpaid Revenue : TMH,2025-08-13 02:18:31,"3,685,237,458","3,685,237,458",0,0,0,0


In [39]:
''' Issue : Postpaid Revenue B2C : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R010100'

issue_postpaid_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.loc[issue_postpaid_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2c_tmh_df = issue_postpaid_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,311,282,032","3,311,282,032","3,305,512,181","3,305,485,985","3,305,485,985",0
1,202402,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,285,072,468","3,285,072,468","3,279,470,827","3,279,449,587","3,279,449,587",0
2,202403,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,254,097,222","3,254,097,222","3,248,644,556","3,248,622,787","3,248,622,787",0
3,202404,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,280,178,978","3,280,178,978","3,274,715,195","3,274,699,228","3,274,699,228",0
4,202405,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,273,314,626","3,273,314,626","3,267,800,934","3,267,780,094","3,267,780,094",0
5,202406,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,284,957,272","3,284,957,272","3,278,609,892","3,278,595,142","3,278,595,142",0
6,202407,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,269,232,707","3,269,232,707","3,264,050,341","3,264,050,142","3,264,050,142",0
7,202408,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,283,314,789","3,283,314,789","3,278,134,554","3,278,134,355","3,278,134,355",0
8,202409,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,281,480,710","3,281,480,710","3,276,410,628","3,276,410,449","3,276,410,449",0
9,202410,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"3,272,186,553","3,272,186,553","3,266,340,594","3,266,340,415","3,266,340,415",0


In [40]:
''' Issue : Postpaid Revenue B2B : TMH (Jan-Jun 2024) '''

v_metict_cd = 'TB2R020100'

issue_postpaid_b2b_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metict_cd]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.loc[issue_postpaid_b2b_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_b2b_tmh_df = issue_postpaid_b2b_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)

issue_postpaid_b2b_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,C,P,G,H,HH,DIFF (C-P)
0,202401,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"411,274,423","411,274,423",0,0,0,0
1,202402,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"401,212,758","401,212,758",0,0,0,0
2,202403,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"403,998,615","403,998,615",0,0,0,0
3,202404,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"412,557,917","412,557,917",0,0,0,0
4,202405,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"402,143,280","402,143,280",0,0,0,0
5,202406,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"428,007,316","428,007,316",0,0,0,0
6,202407,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"398,554,461","398,554,461",0,0,0,0
7,202408,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"397,407,274","397,407,274",0,0,0,0
8,202409,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"424,811,546","424,811,546",0,0,0,0
9,202410,Postpaid,TB2R020100,Postpaid Revenue B2B : TMH,2025-08-13 02:18:31,"413,050,904","413,050,904",0,0,0,0


## ** Current Issue

In [41]:
''' Daily : Postpaid Revenue B2C : TMH '''

v_metict_cd = 'TB2R010100'
# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

issue_daily_post_rev_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.loc[issue_daily_post_rev_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rev_b2c_tmh_df = issue_daily_post_rev_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rev_b2c_tmh_df_display = issue_daily_post_rev_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_rev_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rev_b2c_tmh_df_display[col] = issue_daily_post_rev_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rev_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202508,20250812,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
1,202508,20250811,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
2,202508,20250810,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"421,762,492","421,760,041","421,760,041","421,760,041"
3,202508,20250809,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
4,202508,20250808,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
5,202508,20250807,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
6,202508,20250806,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,"296,575,985","296,575,135","296,575,135","296,575,135"
7,202508,20250805,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
8,202508,20250804,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
9,202508,20250803,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-08-13 05:17:31,nan,nan,nan,nan
